In [1]:
import numpy as np
import ot
from scipy.sparse import coo_matrix
import time
import psutil
import os
import gc

def get_memory_mb():
    """Get current process memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024


[KeOps] Warning : There were warnings or errors :
/bin/sh: brew: command not found

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.

[KeOps] Warning : There were warnings or errors :
/bin/sh: brew: command not found

[KeOps] Warning : OpenMP library not found, it must be downloaded through Homebrew for apple Silicon chips
[KeOps] Warning : OpenMP support is not available. Disabling OpenMP.


## Test: Identity Transport

In [2]:
n = 1000

a = np.ones(n) / n
b = np.ones(n) / n

rows = np.arange(n)
cols = np.arange(n)
data = np.ones(n) 
M_sparse = coo_matrix((data, (rows, cols)), shape=(n, n))

large_cost = 1e6
M_dense = np.full((n, n), large_cost)
np.fill_diagonal(M_dense, 1)

print(f"Sparse edges: {M_sparse.nnz:,} (density={100*M_sparse.nnz/(n*n):.2f}%)")

Sparse edges: 1,000 (density=0.10%)


In [4]:
print("Dense solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_dense, log_dense = ot.emd(a, b, M_dense, log=True)
t_dense = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_dense = mem_after - mem_before

print(f"  Cost: {log_dense['cost']:.10f}")
print(f"  Time: {t_dense*1000:.1f} ms")
print(f"  Memory: {mem_dense:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_dense['warning'] or 'OPTIMAL'}")
print(f"  Flow on diagonal: {np.sum(np.diag(G_dense)):.6f} (expected: 1.0)")
print(f"  Flow off diagonal: {np.sum(G_dense) - np.sum(np.diag(G_dense)):.10f} (expected: 0.0)")

print("Sparse solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_sparse, log_sparse = ot.emd(a, b, M_sparse, log=True)
t_sparse = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_sparse = mem_after - mem_before

print(f"  Cost: {log_sparse['cost']:.10f}")
print(f"  Time: {t_sparse*1000:.1f} ms")
print(f"  Memory: {mem_sparse:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_sparse['warning'] or 'OPTIMAL'}")
if G_sparse is not None:
    print(f"  Flow on diagonal: {np.sum(np.diag(G_sparse)):.6f} (expected: 1.0)")
print()

if log_dense['warning'] is None and log_sparse['warning'] is None:
    diff = abs(log_dense['cost'] - log_sparse['cost'])
    print(f"Cost difference: {diff:.2e}")
    if diff < 1e-6:
        print("SUCCESS: Dense and sparse give identical results!")
        print(f"   Speedup: {t_dense/t_sparse:.2f}x")
        print(f"   Memory savings: {mem_dense - mem_sparse:.2f} MB")
    else:
        print(f"FAILURE: Costs differ by {diff}")

Dense solver:
  Cost: 1.0000000000
  Time: 23.2 ms
  Memory: +11.70 MB (before: 723.6 MB, after: 735.3 MB)
  Result: OPTIMAL
  Flow on diagonal: 1.000000 (expected: 1.0)
  Flow off diagonal: -0.0000000000 (expected: 0.0)
Sparse solver:
  Cost: 1.0000000000
  Time: 0.5 ms
  Memory: +0.14 MB (before: 735.3 MB, after: 735.4 MB)
  Result: OPTIMAL

Cost difference: 0.00e+00
SUCCESS: Dense and sparse give identical results!
   Speedup: 46.84x
   Memory savings: 11.56 MB


## Test : Circular 

In [6]:
n = 25000

a = np.ones(n) / n
b = np.ones(n) / n

rows = []
cols = []
data = []

for i in range(n):
    rows.append(i)
    cols.append(i)
    data.append(1.0)
    
    rows.append(i)
    cols.append((i + 1) % n)
    data.append(1.0)

M_sparse = coo_matrix((data, (rows, cols)), shape=(n, n))

large_cost = 1e6
M_dense = np.full((n, n), large_cost)
for i in range(n):
    M_dense[i, i] = 1.0
    M_dense[i, (i + 1) % n] = 1.0

print(f"Sparse edges: {M_sparse.nnz:,} (density={100*M_sparse.nnz/(n*n):.2f}%)")

Sparse edges: 50,000 (density=0.01%)


In [7]:

print("Dense solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_dense, log_dense = ot.emd(a, b, M_dense, log=True)
t_dense = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_dense = mem_after - mem_before

print(f"  Cost: {log_dense['cost']:.10f}")
print(f"  Time: {t_dense*1000:.1f} ms")
print(f"  Memory: {mem_dense:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_dense['warning'] or 'OPTIMAL'}")
del G_dense, M_dense
gc.collect()
gc.collect()
gc.collect()


Dense solver:
  Cost: 1.0000000000
  Time: 61141.4 ms
  Memory: -643.88 MB (before: 1346.3 MB, after: 702.4 MB)
  Result: OPTIMAL


0

In [9]:
print("Sparse solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_sparse, log_sparse = ot.emd(a, b, M_sparse, log=True)
t_sparse = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_sparse = mem_after - mem_before

print(f"  Cost: {log_sparse['cost']:.10f}")
print(f"  Time: {t_sparse*1000:.1f} ms")
print(f"  Memory: {mem_sparse:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_sparse['warning'] or 'OPTIMAL'}")
print()

if log_dense['warning'] is None and log_sparse['warning'] is None:
    diff = abs(log_dense['cost'] - log_sparse['cost'])
    print(f"Cost difference: {diff:.2e}")
    if diff < 1e-6:
        print("   SUCCESS: Dense and sparse give identical results!")
        print(f"   Speedup: {t_dense/t_sparse:.2f}x")
        print(f"   Memory savings: {mem_dense - mem_sparse:.2f} MB")
    else:
        print(f"    FAILURE: Costs differ by {diff}")

Sparse solver:
  Cost: 1.0000000000
  Time: 8.7 ms
  Memory: +15.53 MB (before: 607.4 MB, after: 623.0 MB)
  Result: OPTIMAL

Cost difference: 0.00e+00
   SUCCESS: Dense and sparse give identical results!
   Speedup: 7046.38x
   Memory savings: -659.41 MB


## Test 3: Random Sparse Bipartite

**Problem**: n sources, n targets, random sparse edges.

**Setup**:
- Each source connects to k random targets
- Every target has at least one incoming edge (guaranteed by construction)

**Feasibility**: NOT guaranteed!
- Connectivity is necessary but NOT sufficient
- With k=20 and uniform distributions, it's VERY LIKELY feasible
- But flow bottlenecks could theoretically still occur
- If infeasible, we'll see it in the results

In [10]:
n = 500
k = 20  # Each source connects to k random targets
np.random.seed(42)

print(f"\nTest 3: Random Sparse Bipartite (n={n}, k={k})")
print("="*70)

# Uniform distributions
a = np.ones(n) / n
b = np.ones(n) / n

# Create random sparse graph: ensure every target is reachable
rows = []
cols = []
data = []

# First, ensure every target is connected to at least one source
for j in range(n):
    i = np.random.randint(0, n)
    cost = np.random.uniform(0.1, 1.0)
    rows.append(i)
    cols.append(j)
    data.append(cost)

# Then add random edges for each source
for i in range(n):
    # Pick k random targets
    targets = np.random.choice(n, size=k, replace=True)
    for j in targets:
        cost = np.random.uniform(0.1, 1.0)
        rows.append(i)
        cols.append(j)
        data.append(cost)

M_sparse = coo_matrix((data, (rows, cols)), shape=(n, n))
M_sparse.sum_duplicates()  # Combine duplicate edges

# Create dense matrix
large_cost = 1e6
M_dense = np.full((n, n), large_cost)
M_sparse_array = M_sparse.toarray()
M_dense[M_sparse_array > 0] = M_sparse_array[M_sparse_array > 0]

print(f"Sparse edges: {M_sparse.nnz:,} (density={100*M_sparse.nnz/(n*n):.2f}%)")
print(f"Edges per source: avg={M_sparse.nnz/n:.1f}")

# Check connectivity
from scipy.sparse import csr_matrix
M_csr = csr_matrix(M_sparse)
rows_with_edges = (M_csr.getnnz(axis=1) > 0).sum()
cols_with_edges = (M_csr.getnnz(axis=0) > 0).sum()
print(f"Sources with edges: {rows_with_edges}/{n}")
print(f"Targets reachable: {cols_with_edges}/{n}")
print()


Test 3: Random Sparse Bipartite (n=500, k=20)
Sparse edges: 10,298 (density=4.12%)
Edges per source: avg=20.6
Sources with edges: 500/500
Targets reachable: 500/500



In [12]:
# Solve with dense
print("Dense solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_dense, log_dense = ot.emd(a, b, M_dense, log=True)
t_dense = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_dense = mem_after - mem_before

print(f"  Cost: {log_dense['cost']:.10f}")
print(f"  Time: {t_dense*1000:.1f} ms")
print(f"  Memory: {mem_dense:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_dense['warning'] or 'OPTIMAL'}")
print()

# Solve with sparse
print("Sparse solver:")
mem_before = get_memory_mb()
t0 = time.perf_counter()
G_sparse, log_sparse = ot.emd(a, b, M_sparse, log=True)
t_sparse = time.perf_counter() - t0
mem_after = get_memory_mb()
mem_sparse = mem_after - mem_before

print(f"  Cost: {log_sparse['cost']:.10f}")
print(f"  Time: {t_sparse*1000:.1f} ms")
print(f"  Memory: {mem_sparse:+.2f} MB (before: {mem_before:.1f} MB, after: {mem_after:.1f} MB)")
print(f"  Result: {log_sparse['warning'] or 'OPTIMAL'}")
print()

# Compare
if log_dense['warning'] is None and log_sparse['warning'] is None:
    diff = abs(log_dense['cost'] - log_sparse['cost'])
    rel_err = diff / log_dense['cost'] * 100 if log_dense['cost'] > 0 else 0
    
    print(f"Dense cost:  {log_dense['cost']:.10f}")
    print(f"Sparse cost: {log_sparse['cost']:.10f}")
    print(f"Difference: {diff:.2e} ({rel_err:.6f}%)")
    print()
    
    if diff < 1e-6:
        print("✅ SUCCESS: Dense and sparse give identical results!")
        print(f"   Speedup: {t_dense/t_sparse:.2f}x")
        print(f"   Memory savings: {mem_dense - mem_sparse:.2f} MB")
    elif rel_err < 0.01:
        print("✅ PASS: Results match within numerical precision")
        print(f"   Speedup: {t_dense/t_sparse:.2f}x")
        print(f"   Memory savings: {mem_dense - mem_sparse:.2f} MB")
    else:
        print(f"❌ FAILURE: Costs differ by {rel_err:.4f}%")
else:
    print(f"❌ FAILURE: One solver failed")
    if log_dense['warning']:
        print(f"   Dense: {log_dense['warning']}")
    if log_sparse['warning']:
        print(f"   Sparse: {log_sparse['warning']}")

Dense solver:
  Cost: 0.1676211637
  Time: 13.2 ms
  Memory: +2.14 MB (before: 631.8 MB, after: 634.0 MB)
  Result: OPTIMAL

Sparse solver:
  Cost: 0.1676211637
  Time: 2.3 ms
  Memory: +0.55 MB (before: 634.0 MB, after: 634.5 MB)
  Result: OPTIMAL

Dense cost:  0.1676211637
Sparse cost: 0.1676211637
Difference: 0.00e+00 (0.000000%)

✅ SUCCESS: Dense and sparse give identical results!
   Speedup: 5.70x
   Memory savings: 1.59 MB
